# Combined Analysis: Generate Stats DF

Beginnings of statistical analysis based on the timeline stats of users, NOT the clustering word data yet. Also should I get the follows?

In [1]:
import os
import pprint
import tqdm
import json
import glob
import jsonlines
import pprint
import pandas as pd
import pickle
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import sys
from collections import Counter
import subprocess
import datetime
import h5py
from typing import NamedTuple
import re
import statsmodels.api as sm
from collections import defaultdict
from operator import attrgetter
import statsmodels.formula.api as smf
from pandas.plotting import scatter_matrix
import networkx as nx

#load in search hashtags
with open('../references/search_hashtags.txt', 'r') as f:
    search_hashtags = f.readlines()
    search_hashtags = [i.replace('\n', '') for i in search_hashtags]
    search_hashtags = [i.replace('#', '') for i in search_hashtags]
    search_hashtags = [i.lower() for i in search_hashtags]
    search_hashtags.remove('وأناكمان')

class daterange(NamedTuple):
    start: str
    end: str

# Parameters

In [2]:
group_num = 1
hashtag_split = True
ngram_range = '34'
min_user = 100

overwrite = True
read_in = True
for_abm = False

# Load in data and basic preprocessing

In [3]:
# Load in data

# set paths
interactions_hdf5_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/03_processed/interactions.hdf5'

group_folder = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/01_raw/0{group_num}_group/'

FAS_volume_df = pd.read_hdf('/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5', key='plot_data')

num_users_group = len(glob.glob(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/01_raw/0{group_num}_group/timeline*.jsonl'))

# define functions
def load_in_data(interactions_hdf5_file):

    # check if data was saved in multiple parts
    with h5py.File(interactions_hdf5_file, 'r') as f:
        interactions_hdf5_keys = f.keys()
        interactions_hdf5_keys = [i for i in interactions_hdf5_keys if re.search(f'interactions_group_{group_num}_part',i)]
        print(interactions_hdf5_keys)

    if len(interactions_hdf5_keys) > 1:
        print('Multi-part interactions read-in')
        list_df = []
        for i in interactions_hdf5_keys:
            list_df.append(pd.read_hdf(interactions_hdf5_file,i))
        df = pd.concat([i for i in list_df], axis=0)
    else:
        df = pd.read_hdf(interactions_hdf5_file,f'interactions_group_{group_num}')
        print('data read in')

    def convert_in_reply_to(x):
        if x:
            return x[0]
        else:
            return None

    print('converting data format for in_reply_to')
    df['in_reply_to'] = df['in_reply_to'].apply(convert_in_reply_to)

    # remove self replies
    print('removing self_replies')
    df = df[df['author_id']!=df['in_reply_to']]

    def combine_quotes(row):
        if isinstance(row, list):
            return row
        if isinstance(row, str):
            return [str]

    # extract quoted tweet ids
    print('Extracting quote ids')
    df['quotes']= df['quotes'].apply(combine_quotes)
    quoted_tweet_ids = []
    for row in df.itertuples():
        for tweet_id in row.quotes:
            quoted_tweet_ids.append(tweet_id)
    quoted_tweet_ids = list(set(quoted_tweet_ids))

    print('saving to file')
    # save quotes to group location
    assert os.path.isdir(group_folder)
    hydration_filename = os.path.join(group_folder, 'quoted_hydration.txt')
    with open(hydration_filename, 'w') as f:
        for line in quoted_tweet_ids:
            f.write(line)
            f.write('\n')

    return df

def convert_quotes(quote_list, dictionary):

    res = []
    for i in quote_list:
        if i not in dictionary:
            pass
        else:
            res.append(dictionary[i])
    return res

def process_hydrated(group_folder, hydrated_filename):
    assert os.path.isfile(hydrated_filename)
    # collect usernames from hydrated:
    quoted_to_user_id = defaultdict(None)
    with jsonlines.open(hydrated_filename, 'r') as reader:
        for line in reader:
            for tweet in line['data']:
                quoted_to_user_id[tweet['id']] = tweet['author_id']
    return quoted_to_user_id

def get_group_daterange():
    with h5py.File('/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5', 'r') as f:
        FAS_peaks = f['peak_detections']
        x = f['segments']['selected_ranges'][int(group_num)-1]
        group_date_range = daterange(
            start = x[0].decode(),
            end = x[1].decode()
        )
    return group_date_range

def unit_conv(val):
    return datetime.datetime.strptime('2017-10-16', '%Y-%m-%d') + datetime.timedelta(days=int(val))

def reverse_unit_conv(date):
    return (datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.datetime.strptime('2017-10-16', '%Y-%m-%d')).days

def get_peaks_for_group(group_date_range):
    #obtain peak times again

    group_start_index = reverse_unit_conv(group_date_range.start)
    group_end_index = reverse_unit_conv(group_date_range.end)


    with h5py.File('/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5', 'r') as f:
        FAS_peaks = f['peak_detections']
        most_prominent_peaks = {}
        for name, h5obj in FAS_peaks.items():

            peak_locations = h5obj['peak_locations']
            peak_locations = [(i,e) for i,e in enumerate(h5obj['peak_locations']) if (unit_conv(e) > datetime.datetime.strptime(group_date_range.start, '%Y-%m-%d')) and (unit_conv(e) < datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d'))]
            peak_indices = [i[0] for i in peak_locations]
            prominences = [element for index, element in enumerate(h5obj['prominences']) if index in peak_indices]
            if len(prominences) == 0:
                continue
            max_prominence = np.argmax(prominences)
            most_prominent_peaks[name] = unit_conv(peak_locations[max_prominence][1])

    return most_prominent_peaks

def process_interaction(df):
    # get unique counts
    df_authors = set(df['author_id'].unique())

    print('getting internal counts...')
    # generate counts of internal mentions/replies
    df['internal'] = (df['in_reply_to'].isin(df['author_id'])) | (df['mentions'].apply(lambda x: any([i in df_authors for i in x]))) | (df['quoted_user_id'].apply(lambda x: any([i in df_authors for i in x])))

    internal = df['internal'].sum()
    print(f'Internal replies (not self replies): {internal}')
    print(f'Total df length: {len(df)}')
    user_interaction_counts = df[['author_id','tweet_id']].groupby('author_id').count()

    print(user_interaction_counts.describe())

    return df

df = load_in_data(interactions_hdf5_file)

hydrated_filename = os.path.join(group_folder, 'hydration.jsonl')
quoted_to_user_id = process_hydrated(group_folder, hydrated_filename)
df['quoted_user_id'] = df['quotes'].apply(convert_quotes, args=(quoted_to_user_id,))
non_null = len(df[~df['quoted_user_id'].isnull()])
print(f'Number of non-null: {non_null}')


group_date_range = get_group_daterange()
expected_counts = FAS_volume_df[(FAS_volume_df['created_at']<group_date_range.end) & (FAS_volume_df['created_at']>group_date_range.start)].groupby('hashtag').sum()/num_users_group

most_prominent_peaks = get_peaks_for_group(group_date_range)

df = process_interaction(df)

['interactions_group_1_part_0', 'interactions_group_1_part_1']
Multi-part interactions read-in
converting data format for in_reply_to
removing self_replies
Extracting quote ids
saving to file
Number of non-null: 7939920
getting internal counts...
Internal replies (not self replies): 848558
Total df length: 7939920
           tweet_id
count   4216.000000
mean    1883.282732
std     2837.788446
min        1.000000
25%      148.750000
50%      763.000000
75%     2498.250000
max    26390.000000


In [4]:
df.columns

Index(['tweet_id', 'author_id', 'tweet_lang', 'text', 'likes', 'reply_count',
       'quote_count', 'retweet_count', 'created_at', 'in_reply_to', 'mentions',
       'quotes', 'replies', 'contains_hashtags', 'quoted_user_id', 'internal'],
      dtype='object')

In [5]:
for hashtag in search_hashtags:
    df['vocab:#'+hashtag] = df['contains_hashtags'].apply(lambda x: any(hashtag.lower() == item.lower() for item in x))

#columns with vocab: in them
vocab_colnames = [i for i in list(df.columns) if 'vocab:#' in i]

# df['network_participation'] = df[vocab_colnames].sum(axis=1)
# df['multiple_participation'] = df['network_participation']>1

# df_participate = df[['author_id','multiple_participation']].groupby('author_id').sum()

# print(df_participate.describe())

# above_one = (df_participate >= 1).sum()
# # print(f'{above_one} users participate in more than one protest network in interactions')

To match on activity, let's collect their base timelines and see how much they tweeted

To determine the output variables from this continuous time scope we need the peaks again.

Start creating the df needed for stats. Each user needs to have
* normal activity before and after
* hashtagged activity before and after
* number of interactions with (each) protest network
* protest networks participated in (organise by pairs?)
* before and after peaks of each network participation


In [6]:
# get list of hashtags in search ones
df['author_total_hashtags'] = df['contains_hashtags'].apply(lambda x: [i for i in x if i.lower() in search_hashtags])
# get peaks for each hashtag into the pandas
for ht in most_prominent_peaks.keys():
    if ht != 'وأناكمان':
        with_sym_ht = f'#{ht}'
        df[f'peak_{ht}'] = df['created_at'].apply(lambda x: x > most_prominent_peaks[ht].date())

# Interacted users for each user

In [7]:
def combine_mention_replies(row):
    res = []
    if row['mentions'] is not None:
        for i in row['mentions']:
            res.append(i)
    if row['quoted_user_id'] is not None:
        for i in row['quoted_user_id']:
            res.append(i)
    if row['in_reply_to'] is not None:
        assert isinstance(row['in_reply_to'],str)
        res.append(row['in_reply_to'])
    return res

# collect users interacted with
df['interacted_users'] = df.apply(combine_mention_replies,axis=1)

# res = []
# for ht_target in most_prominent_peaks.keys():
#     # print(ht_target)
#     # take only the rows for before the peak. Group by 
#     filtered =  df[df[f'vocab:#{ht_target}']&df['internal']]
#     filtered = filtered.groupby('author_id').agg(interacted_total_users = pd.NamedAgg(column="interacted_users", aggfunc="sum"))
#     filtered['ht'] = ht_target
#     res.append(filtered)

# test_df_int = pd.concat([i for i in res], axis=0)
# int_df = test_df_int.reset_index()

# # only keep rows where users did in fact interact with others
# int_df = int_df[int_df['interacted_total_users'].apply(len)>0]



What about reciprocal interactions as a variable?

In [8]:
# explode the rows of interacted users. purely for the purpose of getting these 
# in their own rows
df_exploded = df.explode('interacted_users', ignore_index=True)

# only get the ones that are internal interactions
df_exploded = df_exploded[df_exploded['internal']]

# iterate over hashtags because we want to group by user ids and hashtags.
res = []
for ht_target in most_prominent_peaks.keys():

    # for each hashtag, filter out the hashtag
    filtered =  df_exploded[df_exploded[f'vocab:#{ht_target}']]

    # then groupby author id and count the number of interactions with each other user.
    filtered = filtered.groupby(['author_id','interacted_users']).agg(weight = pd.NamedAgg(column="tweet_id", aggfunc="count"), interacted_total_users = pd.NamedAgg(column="interacted_users", aggfunc=lambda x: list(set(x))))

    # add back the hashtag iterating over to the resulting df
    filtered['ht'] = ht_target

    # append result
    res.append(filtered)

# concatenate to collect results
df_exploded_int = pd.concat([i for i in res], axis=0).reset_index()

# create graph from resulting edge list
G = nx.from_pandas_edgelist(df_exploded_int, "author_id", "interacted_users", edge_attr = ['weight', 'ht'], create_using = nx.MultiDiGraph())

# nx.draw(G, with_labels = False)

# define function to search for reciprocal in the edgelist
def get_reciprocal(row, G):

    try:

        # data=True allows accessing edge weight
        edgelist = G.edges(row['interacted_users'], data=True)
        res = 0

        # search through edgelist to find the REVERSE, i.e. reciprocal. That is, where target is the source of the initial edgelist
        for _, target, attributes in edgelist:
            if target==row['author_id'] and attributes['ht']==row['ht']:
                res+=attributes['weight']
        return res
    except KeyError:
        return 0

# run reciprocal collection
df_exploded_int['reciprocal'] = df_exploded_int.apply(get_reciprocal, args = (G,), axis=1)

# group by author and hashtag as originally planned
int_df = df_exploded_int.groupby(['author_id', 'ht']).agg(
    weight = pd.NamedAgg(column='weight', aggfunc='sum'),
    reciprocal = pd.NamedAgg(column='reciprocal', aggfunc='sum'),
    interacted_total_users = pd.NamedAgg(column="interacted_total_users", aggfunc="sum")
).reset_index()

# calculate percentages
int_df['percent_reciprocal'] = int_df['reciprocal']/int_df['weight']

# # remove duplicate
# df_exploded_int_grouped['interacted_total_users'] = df_exploded_int_grouped['interacted_total_users'].apply(lambda x: list(set(x)))

In [9]:
int_df

,author_id,ht,weight,reciprocal,interacted_total_users,percent_reciprocal
0,1005675566,timesup,1,0,[15907183],0.000000
1,101142807,metoo,6,0,"[780302207325310977, 925039712783306752]",0.000000
2,101142807,timesup,7,0,"[780302207325310977, 925039712783306752]",0.000000
3,101142807,wetoo,13,11,"[101142807, 947423877277171712]",0.846154
4,10118982,metoo,3,0,"[15754281, 290755398, 941782988819521536]",0.000000
...,...,...,...,...,...,...
3293,987050151159255041,timesup,104,0,"[11744152, 11855772, 1198406491, 119972954, 13...",0.000000
3294,988406744,metoo,1,0,[807095],0.000000
3295,992431609,timesup,1,0,[15588104],0.000000
3296,995054521,metoo,1,0,[16012783],0.000000


In [10]:
int_df.describe()

,weight,reciprocal,percent_reciprocal
count,3298.000000,3298.000000,3298.000000
mean,6.933899,1.341722,0.179460
std,37.142446,6.658394,1.424806
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,6.000000,1.000000,0.041667
max,1969.000000,229.000000,76.333333


# Vocabulary Input: Text in interactions

In [11]:
# idea: use the text in interactions to see if frequency of phrases increase?



# Vocabulary Input: Python Clustering Eval Results

Now for vocabulary input. First load in the relevant files

In [12]:
# load bsc model, features names, csr, and user ('document') ids
bsc_output_dir = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/05_model_output/'
bsc_eval_hdf5 = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/05_model_output/bsc_cluster_eval.hdf5'
mapping_dir = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/'

if hashtag_split:
    bsc_model_list_dict = {'before':{}, 'after':{}}
    feature_mapping_dict = {'before':{}, 'after':{}}
    csr_file_dict = {'before':{}, 'after':{}}

    for before_str in ['before','after']:
        for ht in most_prominent_peaks.keys():
            bsc_model_list_dict[before_str][ht] = glob.glob(os.path.join(bsc_output_dir, f'0{group_num}_group',f'*{ht}_{before_str}.obj'))

            if not os.path.isfile(bsc_model_list_dict[before_str][ht][0]):
                print(bsc_model_list_dict[before_str][ht][0])

            # feature_mapping_dict[before_str][ht] = os.path.join(mapping_dir,f'0{group_num}_group',f'mapping_ngram_{ngram_range}_{ht}_{before_str}.obj')

            # # sanity check
            # if not os.path.isfile(feature_mapping_dict[before_str][ht]):
            #     print(feature_mapping_dict[before_str][ht])

            # csr_file_dict[before_str][ht] = os.path.join(mapping_dir,f'0{group_num}_group/user_count_mat_ngram_{ngram_range}_{ht}_{before_str}.obj')

            # # sanity check
            # if not os.path.isfile(feature_mapping_dict[before_str][ht]):
            #     print(feature_mapping_dict[before_str][ht])

else:
    bsc_model_list = glob.glob(os.path.join(bsc_output_dir,f'0{group_num}_group','*.obj'))
    feature_mapping_file = os.path.join(mapping_dir,f'0{group_num}_group',f'mapping_ngram_{ngram_range}.obj')
    csr_file = os.path.join(mapping_dir,f'0{group_num}_group/user_count_mat_ngram_{ngram_range}.obj')

Assess eval

In [13]:
# with h5py.File(bsc_eval_hdf5, 'r') as f:
#     print(f['group_1']['ngram_34']['min_100']['stilleforopptak']['after']['10'][:])

In [14]:
class single_cluster(NamedTuple):
    n_clusters: int
    n_invalid: int
    total_ncut: float
    norm_ncut: float
    hashtag: str
    before: str

if hashtag_split:

    cluster_evals = {'before':{}, 'after':{}}

    for before_str in ['before','after']:
        for ht in most_prominent_peaks.keys():
            cluster_evals[before_str][ht] = []

            with h5py.File(bsc_eval_hdf5, 'r') as f:
                x = f[f'group_{group_num}'][f'ngram_{ngram_range}'][f'min_{min_user}'][ht][before_str]
                try:
                    for key, h5_data in x.items():

                        # numpy data shape
                        h5_data = h5_data[:,1]

                        total_ncut = np.sum([i for i in h5_data if i > 0])
                        n_invalid = np.sum([i < 0 for i in h5_data])
                        cluster_evals[before_str][ht].append(single_cluster(
                            n_clusters=int(key),
                            n_invalid=n_invalid,
                            total_ncut=total_ncut,
                            norm_ncut=total_ncut/int(key),
                            hashtag=ht,
                            before=before_str
                        ))
                except:
                    print(ht, before_str,key)
                    continue

else:

    cluster_evals = []
    with h5py.File(bsc_eval_hdf5, 'r') as f:
        x = f[f'group_{group_num}'][f'ngram_{ngram_range}'][f'min_{min_user}']
        for key, h5_data in x.items():
            h5_data = h5_data[:,1]
            total_ncut = np.sum([i for i in h5_data if i > 0])
            n_invalid = np.sum([i < 0 for i in h5_data])
            cluster_evals.append(single_cluster(
                n_clusters=int(key),
                n_invalid=n_invalid,
                total_ncut=total_ncut,
                norm_ncut=total_ncut/int(key),
                hashtag="None",
                before="None"
            ))

Visualise cluster evaluation, or save the df

In [15]:
ht_best_cluster_dict = {'before':{},'after':{}}
if hashtag_split:
    for ht in most_prominent_peaks.keys():
        try:
            for i in ['before', 'after']:

                c = cluster_evals[i][ht]

                c = sorted(c, key=lambda x: x.n_clusters)

                x_data = [i.n_clusters for i in c]
                y_data = [i.norm_ncut for i in c]

                y_invalid = [i.n_invalid for i in c]

                # f, ax1 = plt.subplots(figsize=(15,10))
                # ax1.plot(x_data, y_data, color = 'blue')
                # ax2 = ax1.twinx()
                # ax2.plot(x_data, y_invalid, color='red')
                # ax1.set_title(f'NCut evaluation for max clusters on ngram range {ngram_range} and min {min_user}')
                # ax1.set_xlabel('k=cluster hyperparameter')
                # ax1.set_ylabel('Normalised Cut')
                # ax2.set_ylabel('Number of empty clusters')
                best_cluster = sorted(c, key=attrgetter('norm_ncut'), reverse=True)[0]
                # ax1.annotate(
                #     f'({best_cluster.n_clusters}, {best_cluster.norm_ncut:.2f}), n_invalid = {best_cluster.n_invalid}',
                #     (best_cluster.n_clusters, best_cluster.norm_ncut),
                #     textcoords = 'offset points',
                #     xytext = (10,0),
                #     ha = 'left'
                # );

                selected_model = best_cluster.n_clusters
                ht_best_cluster_dict[i][ht]=selected_model
        except:
            print(ht)

else:
    cluster_evals = sorted(cluster_evals, key=lambda x: x.n_clusters)

    x_data = [i.n_clusters for i in cluster_evals]
    y_data = [i.norm_ncut for i in cluster_evals]

    y_invalid = [i.n_invalid for i in cluster_evals]

    f, ax1 = plt.subplots(figsize=(15,10))
    ax1.plot(x_data, y_data, color = 'blue')
    ax2 = ax1.twinx()
    ax2.plot(x_data, y_invalid, color='red')
    ax1.set_title(f'NCut evaluation for max clusters on ngram range {ngram_range} and min {min_user}')
    ax1.set_xlabel('k=cluster hyperparameter')
    ax1.set_ylabel('Normalised Cut')
    ax2.set_ylabel('Number of empty clusters')
    best_cluster = sorted(cluster_evals, key=attrgetter('norm_ncut'), reverse=True)[0]
    ax1.annotate(
        f'({best_cluster.n_clusters}, {best_cluster.norm_ncut:.2f}), n_invalid = {best_cluster.n_invalid}',
        (best_cluster.n_clusters, best_cluster.norm_ncut),
        textcoords = 'offset points',
        xytext = (10,0),
        ha = 'left'
    );

    selected_model = best_cluster.n_clusters

In [16]:
with open(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/0{group_num}_group/best_clusters.obj', 'wb') as f:
    pickle.dump(ht_best_cluster_dict, f)

In [17]:
pp=pprint.PrettyPrinter()
pp.pprint(most_prominent_peaks)
pp.pprint(group_date_range)

{'anakaman': datetime.datetime(2018, 5, 24, 0, 0),
 'balancetonporc': datetime.datetime(2018, 1, 10, 0, 0),
 'cuentalo': datetime.datetime(2018, 4, 30, 0, 0),
 'gamani': datetime.datetime(2018, 1, 26, 0, 0),
 'jotambe': datetime.datetime(2017, 11, 24, 0, 0),
 'memyös': datetime.datetime(2017, 12, 12, 0, 0),
 'metoo': datetime.datetime(2018, 3, 9, 0, 0),
 'moiaussi': datetime.datetime(2018, 1, 15, 0, 0),
 'niere': datetime.datetime(2018, 2, 6, 0, 0),
 'niunamenos': datetime.datetime(2017, 11, 25, 0, 0),
 'nårdansenstopper': datetime.datetime(2017, 12, 12, 0, 0),
 'nårmusikkenstilner': datetime.datetime(2017, 11, 22, 0, 0),
 'quellavoltache': datetime.datetime(2018, 3, 8, 0, 0),
 'ricebunny': datetime.datetime(2018, 3, 31, 0, 0),
 'stilleforopptak': datetime.datetime(2017, 11, 16, 0, 0),
 'timesup': datetime.datetime(2018, 1, 8, 0, 0),
 'tôicũngvậy': datetime.datetime(2017, 12, 3, 0, 0),
 'watashimo': datetime.datetime(2018, 4, 6, 0, 0),
 'wetoo': datetime.datetime(2018, 4, 13, 0, 0),
 '

In [18]:
user_doc_ids = sorted(glob.glob(os.path.join(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/01_raw/0{group_num}_group/','timeline*.jsonl')))
user_doc_ids = [re.split('[_.]',i)[-2] for i in user_doc_ids]
assert len(user_doc_ids)>0


def match_index(search_val, doc_ids):
    try:
        return np.where(doc_ids == search_val)[0].item()
    except:
        return -1

class user_interact_dict(NamedTuple):
    author_id: str
    author_dict_before: dict
    author_dict_after: dict
    ht: str

author_interacted_res=[]

for ht in most_prominent_peaks.keys():

    # open bsc model files:
    try:
        best_cluster_before = ht_best_cluster_dict['before'][ht]
        best_cluster_after  = ht_best_cluster_dict['after'][ht]
    except KeyError:
        print(ht)
        continue


    bsc_before = os.path.join(bsc_output_dir,f'0{group_num}_group/', f'bsc_python_cluster_{best_cluster_before}_ngram_{ngram_range}_min_{min_user}_{ht}_before.obj')
    assert os.path.isfile(bsc_before)
    # logging.debug(f'BSC before is {bsc_before}')
    bsc_after = os.path.join(bsc_output_dir,f'0{group_num}_group/', f'bsc_python_cluster_{best_cluster_after}_ngram_{ngram_range}_min_{min_user}_{ht}_after.obj')
    assert os.path.isfile(bsc_before)
    # logging.debug(f'BSC after is {bsc_after}')


    # logging.debug('Attempting to load in bsc before')
    with open(bsc_before, 'rb') as f:
        model_before = pickle.load(f)
    # logging.debug('Done. Attempting to load in bsc after')
    with open(bsc_after, 'rb') as f:
        model_after = pickle.load(f)

    mask_file = os.path.join(bsc_output_dir,f'0{group_num}_group/',f'mask_bsc_python_cluster_ngram_{ngram_range}_min_{min_user}_{ht}_before.obj')

    with open(mask_file, 'rb') as f:
        mask = pickle.load(f)

    masked_user_doc_ids = np.array(user_doc_ids)[mask]

    # rows_before = 0
    # for i in range(int(best_cluster_before)):
    #     rows_ind, _ = model_before.get_indices(i)
    #     rows_before += rows_ind.shape[0]

    # print(rows_before)
    # print(len(masked_user_doc_ids))



    # logging.debug(f'Filter only to this ht: {ht}')
    int_df_sub = int_df[int_df['ht']==ht]

    for row in int_df_sub.itertuples():

        # logging.debug('gathering indices of where author is in features')
        author_index_before = np.where(masked_user_doc_ids == row.author_id)[0]
        indices_interacted_before = [match_index(i, masked_user_doc_ids) for i in row.interacted_total_users]
        interacted_before = zip(row.interacted_total_users, indices_interacted_before)

        author_index_after = np.where(masked_user_doc_ids == row.author_id)[0]
        indices_interacted_after = [match_index(i, masked_user_doc_ids) for i in row.interacted_total_users]
        interacted_after = zip(row.interacted_total_users, indices_interacted_after)

        author_interacted_before = defaultdict(list)
        author_interacted_after = defaultdict(list)
        if author_index_before.shape[0] > 0:
            for i in range(int(best_cluster_before)):
                model_before_rows = model_before.get_indices(i)[0]
                if author_index_before not in model_before_rows:
                    continue
                else:
                    for user, index in interacted_before:
                        if index in model_before_rows:
                            author_interacted_before[user] = True
                        else:
                            author_interacted_before[user] = False

        if author_index_after.shape[0] > 0:
            for i in range(int(best_cluster_after)):
                model_after_rows = model_after.get_indices(i)[0]
                if author_index_after not in model_after_rows:
                    continue
                else:
                    for user, index in interacted_after:
                        if index in model_after_rows:
                            author_interacted_after[user] = True
                        else:
                            author_interacted_after[user] = False

        author_interacted_res.append(
            user_interact_dict(
                author_id=row.author_id,
                author_dict_before=author_interacted_before,
                author_dict_after=author_interacted_after,
                ht=row.ht
            )
        )

        # for i in author_interacted_before.keys():
        #     if not author_interacted_before[i]:
        #         if author_interacted_after[i]:
        #             print(row.author_id, i, author_interacted_before[i], author_interacted_after[i])

        # int_df.loc[row.author_id,'res_before']=author_interacted_before
        # int_df.loc[row.author_id,'res_after']=author_interacted_after

# combine results into dataframe

int_cluster_df = pd.DataFrame(columns = ['author_id', 'interact_user', 'ht', 'same_cluster_before', 'same_cluster_after'])

def get_author_dict_bool(d,key):
    if d[key] == []:
        return False
    else:
        return d[key]

for i in author_interacted_res:
    for j,k in i.author_dict_before.items():
        int_cluster_df.loc[len(int_cluster_df.index)] = [i.author_id, j, i.ht, k, get_author_dict_bool(i.author_dict_after,j)]

Load in selected model data, csr, etc.

In [19]:
# def model_params_from_file(filename):
#     return re.split('[_.]',filename)

# selected_model_file = [i for i in bsc_model_list if int(model_params_from_file(i)[-6])==selected_model and model_params_from_file(i)[-4] == ngram_range][0]

# with open(selected_model_file, 'rb') as f:
#     model = pickle.load(f)

# # open mapping also get relevent features names and user ids
# with open(feature_mapping_file, 'rb') as f:
#     feature_names = pickle.load(f)

# # also load in csr
# with open(csr_file,'rb') as f:
#     csr = pickle.load(f)

# # get also the userlist that was used to generate this for 'document' names
# user_doc_ids = sorted(glob.glob(os.path.join(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/01_raw/0{group_num}_group/','timeline*.jsonl')))
# user_doc_ids = [re.split('[_.]',i)[-2] for i in user_doc_ids]
# assert len(user_doc_ids)>0
# # function to extract top phrases from cluster
# def extract_cluster_words(feature_names, user_ids, csr, model, cluster_num, top_n = 20):
#     row_ind, col_ind = model.get_indices(cluster_num)

#     csr_cluster = csr[row_ind][:,col_ind]
#     feature_csr_sums = csr_cluster.sum(axis=0)
#     top_features = np.argsort(feature_csr_sums)
#     top_features = np.asarray(top_features).flatten()
#     return [feature_names[i] for i in top_features[-1:-top_n:-1]]

# cluster_words_dict = {}

# for i in range(best_cluster.n_clusters):

#     top_n = 50

#     cluster_words_dict[i] = extract_cluster_words(
#         feature_names,
#         user_doc_ids,
#         csr,
#         model,
#         i,
#         top_n = top_n
#     )

#     if len(cluster_words_dict[i]) < top_n-1:
#         cluster_words_dict[i] = cluster_words_dict[i] + [None]*(top_n-1-len(cluster_words_dict[i]))

In [20]:
# df_cluster_words = pd.DataFrame.from_dict(cluster_words_dict)

# Finally, a statistical model

### Activity

In [21]:
# FAS analysis file
FAS_peak_analysis_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5'

# insert activity of users
def date_to_array_index(date, daterange):
    return (date - datetime.datetime.strptime(daterange.start, '%Y-%m-%d')).days

activity_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/03_processed/activity_counts.hdf5'

with h5py.File(activity_file, 'r') as f:
    # print(f.keys())
    x = f['group_1']['1005675566']['hashtagged'].attrs['feature_order']
    ht_row_mapping = x.split(';')

In [22]:
# do by hashtag. group by users and then whether it was before peak
class stats_df_tuple(NamedTuple):
    ht_target: str
    data: pd.DataFrame

def retrieve_activity_user(row, daterange=None, activity_file = None, activity='hashtagged', hashtag=None, group_num = 1, ht_row_mapping = None, averaged = False):

    try:
        if activity == 'hashtagged':
            if row[f'peak_{hashtag}']:
                after = date_to_array_index(most_prominent_peaks[hashtag],daterange)
                # get ht row index
                assert hashtag in ht_row_mapping
                ht_index = ht_row_mapping.index(hashtag)
                # print(f'group_{group_num}', row['author_id'], after)
                result = activity_file[f'group_{group_num}'][row['author_id']][activity][ht_index,after:]
            else:
                up_to = date_to_array_index(most_prominent_peaks[hashtag], daterange)
                # get ht row index
                assert hashtag in ht_row_mapping
                ht_index = ht_row_mapping.index(hashtag)
                # print(f'group_{group_num}', row['author_id'], ht_index, up_to)
                result = activity_file[f'group_{group_num}'][row['author_id']][activity][ht_index,:up_to]
        else:
            if row[f'peak_{hashtag}']:
                after = date_to_array_index(most_prominent_peaks[hashtag], daterange)
                # print(f'group_{group_num}', row['author_id'], after)
                result = activity_file[f'group_{group_num}'][row['author_id']][activity][after:]
            else:
                up_to = date_to_array_index(most_prominent_peaks[hashtag], daterange)
                # print(f'group_{group_num}', row['author_id'], up_to)
                result = activity_file[f'group_{group_num}'][row['author_id']][activity][:up_to]

        if len(result)>0:
            if averaged:
                s = np.sum(result)/len(result)
                return s
            else:
                s = np.sum(result)
                # print(s)
                return s
        else:
            return 0
    except:
        print('not found')
        return 0

def generate_stats_df_components(df, for_abm = False,
        most_prominent_peaks = most_prominent_peaks,
        activity_file=activity_file,
        group_date_range=group_date_range,
        group_num = group_num,
        ht_row_mapping = ht_row_mapping,
    ):
    res = []
    with h5py.File(activity_file, 'r') as f:
        for ht_target in most_prominent_peaks.keys():
            # print(ht_target)
            # take only the rows for before the peak. Group by 
            if for_abm:
                filtered =  df[~df[f'peak_{ht_target}']]
            else:
                filtered =  df[~df[f'peak_{ht_target}'] & df[f'vocab:#{ht_target}']]
            num_interacts = filtered.groupby(['author_id']).agg(int_pre_peak = pd.NamedAgg(column="tweet_id", aggfunc="count"))
            act_pre_peak = filtered.sort_values('created_at', ascending=False).groupby('author_id').agg(lambda x: x.iloc[0]).reset_index()

            if act_pre_peak.empty:
                num_interacts['act_pre_peak'] = 0
                num_interacts['act_pre_peak_avg'] = 0
                num_interacts['norm_act_pre_peak'] = 0
                num_interacts['norm_act_pre_peak_avg'] = 0
            else:
                act_pre_peak['act_pre_peak'] = act_pre_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    hashtag=ht_target,
                    group_num=group_num,
                    ht_row_mapping = ht_row_mapping,
                    axis=1
                )

                act_pre_peak['act_pre_peak_avg'] = act_pre_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    hashtag=ht_target,
                    group_num=group_num,
                    ht_row_mapping = ht_row_mapping,
                    averaged=True,
                    axis=1
                )

                # add normal activity too
                act_pre_peak['norm_act_pre_peak'] = act_pre_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    activity='normal',
                    hashtag=ht_target,
                    group_num=group_num,
                    axis=1
                )

                act_pre_peak['norm_act_pre_peak_avg'] = act_pre_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    activity='normal',
                    hashtag=ht_target,
                    group_num=group_num,
                    averaged=True,
                    axis=1
                )

                num_interacts = num_interacts.join(act_pre_peak[['author_id','act_pre_peak', 'norm_act_pre_peak','act_pre_peak_avg', 'norm_act_pre_peak_avg']].set_index('author_id'))

            if for_abm:
                act_post_peak = df[df[f'peak_{ht_target}']]
            else:
                act_post_peak = df[df[f'peak_{ht_target}'] & df[f'vocab:#{ht_target}']]
            act_post_peak = act_post_peak.sort_values('created_at', ascending=False).groupby('author_id').agg(lambda x: x.iloc[0]).reset_index()

            if act_post_peak.empty:
                num_interacts['act_post_peak'] = 0
                num_interacts['act_post_peak_avg'] = 0
            else:
                act_post_peak['act_post_peak'] = act_post_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    hashtag=ht_target,
                    group_num=group_num,
                    ht_row_mapping = ht_row_mapping,
                    axis=1
                )

                act_post_peak['act_post_peak_avg'] = act_post_peak.apply(
                    retrieve_activity_user,
                    daterange=group_date_range,
                    activity_file=f,
                    hashtag=ht_target,
                    group_num=group_num,
                    ht_row_mapping = ht_row_mapping,
                    averaged=True,
                    axis=1
                )

                temp = act_post_peak[['author_id','act_post_peak', 'act_post_peak_avg']].set_index('author_id')

                num_interacts = num_interacts.join(temp)
            num_interacts = num_interacts.reset_index().fillna(0)
            num_interacts['ht'] = ht_target
            res.append(stats_df_tuple(ht_target=ht_target, data=num_interacts))
    return res

res = generate_stats_df_components(df, for_abm = for_abm)

In [23]:
stats_df = pd.concat([i.data for i in res], axis=0)

In [24]:
# add primary ht back into stats df
# stats_df['primary_ht'] = df['author_id'].apply(process_primary_ht, args = (primary_ht,expected_counts,))

In [25]:
stats_df.describe()

,int_pre_peak,act_pre_peak,act_pre_peak_avg,norm_act_pre_peak,norm_act_pre_peak_avg,act_post_peak,act_post_peak_avg
count,2726.000000,2726.000000,2726.000000,2726.000000,2726.000000,2726.000000,2726.000000
mean,29.019076,83.849963,0.774565,2352.147836,23.098917,32.505503,0.329629
std,255.639394,434.945339,3.825799,2786.310440,26.280607,104.381649,1.162335
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,19.250000,0.219298,508.000000,5.342105,0.000000,0.000000
50%,4.000000,44.000000,0.412281,1415.000000,14.460526,13.000000,0.144578
75%,17.000000,77.000000,0.703216,3101.250000,30.922515,33.000000,0.337349
max,12260.000000,20189.000000,177.096491,25099.000000,220.166667,3372.000000,40.626506


In [26]:
len(stats_df['author_id'].unique())

2120

## Incorporate Vocab

In [27]:
# # first attempt 2021-12-01: Simply assign cluster assignment as categorical variable

# # get rows that correspond to the users
# author_model_indices = np.searchsorted(user_doc_ids,stats_df['author_id'])
# cluster_labels = model.row_labels_[author_model_indices]

# stats_df['cluster'] = cluster_labels
# stats_df['cluster'] = stats_df['cluster'].astype('category')

## Incorporate likes

Though experimentation is not easy to operationalise, perhaps one way of realising a small experimentation is the feedback on individual tries ('tweets') with substantial likes.

In [ ]:
df.columns

In [29]:
df['likes_deviation'] = df.groupby('author_id')['likes'].transform(lambda x: abs(x - x.mean()) / x.std())
df['likes_std_2'] = df['likes_deviation'] > 2

likes_res = []
for ht_target in most_prominent_peaks.keys():
    # print(ht_target)
    # take only the rows for before the peak. Group by 
    if for_abm:
        filtered =  df[~df[f'peak_{ht_target}']]
    else:
        filtered =  df[~df[f'peak_{ht_target}'] & df[f'vocab:#{ht_target}']]
    likes_std = filtered.groupby(['author_id']).agg(likes_std = pd.NamedAgg(column="likes_std_2", aggfunc="sum"))
    likes_std['ht'] = ht_target
    likes_res.append(likes_std)

likes_df = pd.concat([i for i in likes_res], axis=0).reset_index()

In [29]:
len(likes_df['author_id'].unique())

4211

In [30]:
stats_df = stats_df.merge(likes_df, on=['author_id','ht'], how='left')

In [31]:
len(stats_df['author_id'].unique())

4211

### Process logarithmic

In [32]:
# stats_df['log_act_post_peak'] = np.log(stats_df['act_post_peak']+1)
# stats_df['log_act_pre_peak'] = np.log(stats_df['act_pre_peak']+1)
# stats_df['log_int_pre_peak'] = np.log(stats_df['int_pre_peak']+1)
# stats_df['log_norm_act_pre_peak'] = np.log(stats_df['norm_act_pre_peak']+1)

## Incorporate m3 inference

In [33]:
m3_file = f'../data/03_processed/m3inferred_group_{group_num}.json'
with open(m3_file,'r') as f:
    m3_results = json.load(f)

In [34]:
stats_df['org'] = stats_df['author_id'].apply(lambda x: m3_results[x]['org']['non-org']<=0.5)

def age_m3(id, m3_res):
    d = m3_res[id]['age']
    return max(d, key=d.get)

stats_df['age'] = stats_df['author_id'].apply(age_m3, args=(m3_results,))
stats_df['gender'] = stats_df['author_id'].apply(lambda x: m3_results[x]['gender']['female']>=0.5)

In [35]:
# incorporate predicted language on bio as well
m3_transformed_file = f'../data/01_raw/0{group_num}_group/m3_users_transformed.jsonl'
m3_lang = {}
with jsonlines.open(m3_transformed_file, 'r') as f:
    for line in f:
        m3_lang[line['id']] = line['lang']

stats_df['lang'] = stats_df['author_id'].apply(lambda x: m3_lang[x])

## Incorporate hashtag_lang

In [36]:
hashtag_lang_dict = {
    'metoo': 'en',
    'balancetonporc': 'fr',
    'moiaussi': 'fr',
    'نه_یعنی_نه': 'ar',
    '米兔': 'zh',
    '我也是': 'zh',
    'gamani': 'he',
    'tôicũngvậy': 'vi',
    '私も': 'ja',
    'watashimo': 'ja',
    '나도': 'ko',
    '나도당했다': 'ko',
    'גםאנחנו' : 'he',
    'ятоже': "ru",
    'ricebunny': 'zh',
    'enazeda': "ar",
    'anakaman': "ar",
    'yotambien': 'es',
    'sendeanlat': 'tr' ,
    'kutoo': 'ja',
    'withyou': 'ja',
    'wetoo': 'ja',
    'cuentalo': 'es',
    'quellavoltache': 'it',
    'niunamenos': 'es',
    'woyeshi': 'zh',
    'myharveyweinstein': 'en',
    'noustoutes': 'fr',
    'stilleforopptak': 'no',
    'nårdansenstopper': 'no',
    'nårmusikkenstilner': 'no',
    'memyös': 'fi',
    'timesup': 'en',
    'niere': 'fr',
    'jotambe': 'en'
}

In [37]:
stats_df['hashtag_lang'] = stats_df.apply(lambda x: hashtag_lang_dict[x['ht']], axis=1)
stats_df['lang_diff'] = stats_df.apply(lambda x: x['lang'] != hashtag_lang_dict[x['ht']], axis=1)

# Dependent Variable: Time before tweet in a different language

In [38]:
# perhaps filter out people without lang diff?

def user_lang_diff_interval(user_id, timeline_dir, peak_date, ht, user_lang, ht_lang, num=1):

    if user_lang == ht_lang:
        return np.nan

    # get user file
    filename = os.path.join(timeline_dir, f'timeline_{user_id}.jsonl')
    assert os.path.isfile(filename)

    current_best = datetime.timedelta(days=10000)

    # read through file
    with jsonlines.open(filename, 'r') as reader:
        for tweet_jsonl in reader:
            tweet_list_in_file = tweet_jsonl['data']
            for tweet_data in tweet_list_in_file:
                tweet_created_at = datetime.datetime.fromisoformat(tweet_data['created_at'][:-1])
                if tweet_created_at < peak_date:
                    continue
                elif tweet_data['lang'] != user_lang and tweet_data['lang'] == ht_lang and tweet_created_at - peak_date < current_best:
                    if 'entities' in tweet_data and 'hashtags' in tweet_data['entities']:
                        # print([i['tag'].lower() for i in tweet_data['entities']['hashtags']])
                        # if ht.lower() in [i['tag'].lower() for i in tweet_data['entities']['hashtags']]:
                            # print('bdas')
                        current_best = tweet_created_at - peak_date

    if current_best == datetime.timedelta(days=10000):
        return np.nan

    return current_best

if not for_abm:
    stats_df['interval'] = stats_df.apply(lambda x: user_lang_diff_interval(
        x['author_id'],
        f'../data/01_raw/0{group_num}_group',
        most_prominent_peaks[x['ht']],
        x['ht'],
        x['lang'],
        x['hashtag_lang']
    ), axis=1)

In [39]:
stats_df['interval'][~stats_df['interval'].isnull()].describe()

count                          744
mean     2 days 21:12:47.325268817
std      4 days 04:55:15.855892271
min                0 days 00:00:00
25%         0 days 09:52:55.500000
50%                1 days 00:53:29
75%         3 days 13:18:03.250000
max               24 days 14:45:33
Name: interval, dtype: object

In [40]:
#interval in minutes
stats_df['interval_minutes'] = stats_df['interval'].apply(lambda x: x.total_seconds()/60)

In [41]:
print(len(stats_df['author_id'].unique()))
print(len(df['author_id'].unique()))

3010
9674


# Dependent Variable: Language Clustering

In [39]:
int_cluster_df['same_cluster_change'] = int_cluster_df['same_cluster_after']-int_cluster_df['same_cluster_before']

In [40]:
int_cluster_df_by_author = int_cluster_df.groupby(['ht','author_id'])['same_cluster_change'].mean().reset_index()

In [41]:
int_cluster_df_by_author.describe()

,same_cluster_change
count,3248.000000
mean,-0.215165
std,0.430269
min,-1.000000
25%,-0.500000
50%,0.000000
75%,0.000000
max,1.000000


In [42]:
stats_df

,author_id,int_pre_peak,act_pre_peak,norm_act_pre_peak,act_pre_peak_avg,norm_act_pre_peak_avg,act_post_peak,act_post_peak_avg,ht,likes_std,org,age,gender,lang,hashtag_lang,lang_diff
0,1005675566,1229,0.0,2972,0.0,15.642105,0.0,0.0,anakaman,20,False,>=40,True,en,ar,True
1,101142807,8487,0.0,13456,0.0,70.821053,0.0,0.0,anakaman,392,False,19-29,False,un,ar,True
2,101177558,19,0.0,210,0.0,1.105263,0.0,0.0,anakaman,1,True,19-29,False,un,ar,True
3,10118982,3117,0.0,11782,0.0,62.010526,0.0,0.0,anakaman,42,False,>=40,True,en,ar,True
4,1012146463,4284,0.0,6419,0.0,33.784211,0.0,0.0,anakaman,98,False,30-39,True,es,ar,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109521,99408825,228,0.0,1190,0.0,11.442308,0.0,0.0,나도당했다,2,False,30-39,True,en,ko,True
109522,995054521,274,0.0,214,0.0,2.057692,0.0,0.0,나도당했다,16,True,<=18,False,en,ko,True
109523,995341885,22,0.0,22,0.0,0.211538,0.0,0.0,나도당했다,2,False,19-29,True,un,ko,True
109524,997420128,13,0.0,50,0.0,0.480769,0.0,0.0,나도당했다,1,False,19-29,True,en,ko,True


In [43]:
new_stats_df = stats_df.merge(int_cluster_df_by_author, on=['ht', 'author_id'], how='left')

In [44]:
new_stats_df.describe()

,int_pre_peak,act_pre_peak,norm_act_pre_peak,act_pre_peak_avg,norm_act_pre_peak_avg,act_post_peak,act_post_peak_avg,likes_std,same_cluster_change
count,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,3203.000000
mean,887.690311,3.151224,1656.165038,0.029625,20.325156,2.186896,0.021486,14.535718,-0.217395
std,1630.119353,77.912486,2889.517207,0.686748,27.906952,29.403709,0.277051,31.122847,0.429901
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,53.000000,0.000000,132.000000,0.000000,3.097222,0.000000,0.000000,1.000000,-0.500000
50%,274.000000,0.000000,574.000000,0.000000,10.245614,0.000000,0.000000,4.000000,0.000000
75%,1003.000000,0.000000,1866.000000,0.000000,25.903762,0.000000,0.000000,16.000000,0.000000
max,26378.000000,20189.000000,25099.000000,177.096491,655.400000,5389.000000,40.626506,2013.000000,1.000000


In [45]:
stats_df = new_stats_df

In [46]:
stats_df = stats_df.merge(int_df[['author_id', 'ht', 'weight', 'reciprocal', 'percent_reciprocal']], on=['ht', 'author_id'], how='left')

In [47]:
stats_df

,author_id,int_pre_peak,act_pre_peak,norm_act_pre_peak,act_pre_peak_avg,norm_act_pre_peak_avg,act_post_peak,act_post_peak_avg,ht,likes_std,org,age,gender,lang,hashtag_lang,lang_diff,same_cluster_change,weight,reciprocal,percent_reciprocal
0,1005675566,1229,0.0,2972,0.0,15.642105,0.0,0.0,anakaman,20,False,>=40,True,en,ar,True,NaN,NaN,NaN,NaN
1,101142807,8487,0.0,13456,0.0,70.821053,0.0,0.0,anakaman,392,False,19-29,False,un,ar,True,NaN,NaN,NaN,NaN
2,101177558,19,0.0,210,0.0,1.105263,0.0,0.0,anakaman,1,True,19-29,False,un,ar,True,NaN,NaN,NaN,NaN
3,10118982,3117,0.0,11782,0.0,62.010526,0.0,0.0,anakaman,42,False,>=40,True,en,ar,True,NaN,NaN,NaN,NaN
4,1012146463,4284,0.0,6419,0.0,33.784211,0.0,0.0,anakaman,98,False,30-39,True,es,ar,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109521,99408825,228,0.0,1190,0.0,11.442308,0.0,0.0,나도당했다,2,False,30-39,True,en,ko,True,NaN,NaN,NaN,NaN
109522,995054521,274,0.0,214,0.0,2.057692,0.0,0.0,나도당했다,16,True,<=18,False,en,ko,True,NaN,NaN,NaN,NaN
109523,995341885,22,0.0,22,0.0,0.211538,0.0,0.0,나도당했다,2,False,19-29,True,un,ko,True,NaN,NaN,NaN,NaN
109524,997420128,13,0.0,50,0.0,0.480769,0.0,0.0,나도당했다,1,False,19-29,True,en,ko,True,NaN,NaN,NaN,NaN


In [48]:
stats_df.describe()

,int_pre_peak,act_pre_peak,norm_act_pre_peak,act_pre_peak_avg,norm_act_pre_peak_avg,act_post_peak,act_post_peak_avg,likes_std,same_cluster_change,weight,reciprocal,percent_reciprocal
count,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,109526.000000,3203.000000,3207.000000,3207.000000,3207.000000
mean,887.690311,3.151224,1656.165038,0.029625,20.325156,2.186896,0.021486,14.535718,-0.217395,6.815404,1.330839,0.180657
std,1630.119353,77.912486,2889.517207,0.686748,27.906952,29.403709,0.277051,31.122847,0.429901,37.282380,6.682254,1.444011
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,0.000000
25%,53.000000,0.000000,132.000000,0.000000,3.097222,0.000000,0.000000,1.000000,-0.500000,1.000000,0.000000,0.000000
50%,274.000000,0.000000,574.000000,0.000000,10.245614,0.000000,0.000000,4.000000,0.000000,2.000000,0.000000,0.000000
75%,1003.000000,0.000000,1866.000000,0.000000,25.903762,0.000000,0.000000,16.000000,0.000000,6.000000,1.000000,0.041667
max,26378.000000,20189.000000,25099.000000,177.096491,655.400000,5389.000000,40.626506,2013.000000,1.000000,1969.000000,229.000000,76.333333


In [49]:
# new_stats_df[~new_stats_df['lang_diff'].isnull() & ~new_stats_df['same_cluster_change'].isnull()]

In [50]:
# new_stats_df = new_stats_df[~new_stats_df['lang_diff']]

# Save Progress

In [51]:
# # save progress
# combined_analysis_save_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/combined_analysis.hdf5'

# if os.path.isfile(combined_analysis_save_file):
#     with h5py.File(combined_analysis_save_file, 'a') as f:
#         check = f'group_{group_num}' in f.keys()
#         if check and overwrite:
#             print('overwriting')
#             del f[f'group_{group_num}']
#             stats_df.to_hdf(combined_analysis_save_file,key=f'group_{group_num}', format='table')
#         elif check and read_in:
#             print('reading in')
#             stats_df = pd.read_hdf(combined_analysis_save_file, f'group_{group_num}')
#         else:
#             print('key does not yet exist. Writing...')
#             stats_df.to_hdf(combined_analysis_save_file,key=f'group_{group_num}', format='table')

# else:
#     print('saving to new file')
#     stats_df.to_hdf(combined_analysis_save_file,key=f'group_{group_num}', format='table')

In [52]:
stats_df_save_dir = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/'
if for_abm:
    save_filename = os.path.join(stats_df_save_dir, f'ABM_stats_df_group_{group_num}.obj')
else:
    save_filename = os.path.join(stats_df_save_dir, f'stats_df_group_{group_num}.obj')

if os.path.isfile(save_filename) and overwrite:
    with open(save_filename, 'wb') as f:
        pickle.dump(stats_df, f)
    print('overwriting')
elif os.path.isfile(save_filename) and read_in:
    with open(save_filename, 'rb') as f:
        stats_df = pickle.load(f)
    print('reading in')
else:
    with open(save_filename, 'wb') as f:
        pickle.dump(stats_df, f)
    print('saving')

overwriting


In [53]:
# save df raw
stats_df_save_dir = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/'
if for_abm:
    save_filename = os.path.join(stats_df_save_dir, f'ABM_raw_df_group_{group_num}.obj')
else:
    save_filename = os.path.join(stats_df_save_dir, f'raw_df_group_{group_num}.obj')

if os.path.isfile(save_filename) and overwrite:
    with open(save_filename, 'wb') as f:
        pickle.dump(df, f)
    print('overwriting')
else:
    with open(save_filename, 'wb') as f:
        pickle.dump(df, f)
    print('saving')


overwriting


# Model Selection

In [57]:
def forward_selected(data, response, family=sm.families.family.NegativeBinomial(alpha=2)):
    '''Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    '''

    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            try:
                score = smf.glm(formula, data, family=family).fit().aic
            except ValueError:
                continue
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.glm(formula, data, family=family).fit()
    return model

# Further features:

Revisiting Chabot (2010):

![Diffusion](diffusion.png)

* Awareness is hypothesised by Chabot to be through mass media, for example. Not operationalisable here and not really of interest either.
* Translation is the 'meaningful' dialogue between potential transmitters and receivers. This is the key thing to operationalise and why I wanted clustering in the first place
    * perhaps manual coding of key phrases
    * feature measuring the 
* Experimentation
    * This is extremely difficult to model - how can I infer from tweets whether they are 'testing' something out?
    * Theory needs developing here for internet age - Chabot writes '[experimentation] begins with small-scale direct action campaigns and implies intensive dialogue among adoption pioneers'
        * potential operationalisation - measuring more intense periods of interaction WITHIN a hashtag version before peak
        * but small-scale direct action campaigns are not really operationalisable in this context.
    * or another mechanism is likes on one tweet in one interaction.
* Movement Application -> this is my outcome variable, operationalised as amount of integration into target hashtag version
    * another measure - interval before first tweet in another language?
    * make lots of scatter plots
    * scattermatrix pandas
    * more GLMs

* Over and above all of this, who is an activist?
    * but should we even make that distinction? and is that one for me to make? no
    * maybe it made sense in collective action models to differentiate between those who participate in protests, for example, but doing this online is less clear.
    * That's not to say we can't make meaningful segmentations, though. Could do Verified/not verified (there's literature to suggest that's a meaningful difference), and perhaps
    * **think about separating by organisation**
    * just draw line on activity? threshold. Can vary parameter and check robustness obviously.
    * That would be an honest way to separate the users

Next steps:

* expand measure of activity to not be a simple aggregate of before and after a peak. Perhaps a few days before and a few days after? Segment time into weeks?
    * or some measure of density of interactions
* cluster without hashtags?


From Earl (2010):

![Earl](Earl-2010.png)

Earl contends that there social movement scholars haven't studied second order effects of diffusion, but doesn't really discuss the diffusion mechanics itself

From Penney and Dadas (2014) -> rapid distribution of text through Twitter allowed protestors to buidl a geeographically dispersed, netwroked counterpublic. This is captured by the timed clusters.

However, their interviews also stress the difficulty of using Twitter to engage in discussion as it gets frustrating and debate isn't meaningful.

They also mention how Twiter helped strengthen ties but not necessarily across countries, and Occupy also had more of an offline component. 

NOTE TO SELF: USE TWITTER PLATFORM AFFORANCES TO GENERATE HYPOTHESIS
* e.g. think of Massanari (2017).
* Draw from harassment literature (ask Alexa). Women who get harassed online -> less likely to interact or to participate?


From Vasi Suh (2016):
* they also do stats models